In [1]:
#!pip install pomegranate

In [2]:
#!pip install pymc3

In [3]:
#!pip install graphviz

In [4]:
#!pip install pygraphviz

In [5]:
#!pip install matplotlib

In [6]:
#!pip install openpyxl

In [7]:
#!pip install sklearn

In [1]:
#imports
import pandas as pd
import numpy as np
import arviz as az
import pymc3 as pm
import matplotlib.pyplot as plt
from pomegranate import *

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [22]:
#read in data
pd.set_option('display.max_columns', None)

#name format is df survey#_wave# if resend, add (_2 at end)

df1_1 = pd.read_csv("survey1.1.csv", encoding='cp1252')
df1_1_2 = pd.read_csv("survey1.1.2.csv", encoding='cp1252')
df1_2 = pd.read_csv("survey1.2.csv", encoding ='cp1252')
#survey 1 wave 3
#survey1.3.csv
#df1_3 = pd.read_csv...
df2_1 = pd.read_csv("survey2.1.csv", encoding='cp1252')
df2_1_2 = pd.read_csv("survey2.1.2.csv", encoding='cp1252')
df2_2 = pd.read_csv("survey2.2.csv", encoding='cp1252')
df3_1 = pd.read_csv("survey3.1.csv", encoding='cp1252')
print(df2_2.shape)

(333, 70)


In [23]:
survey1 =pd.concat([df1_1, df1_1_2, df1_2]) #add df1_3
survey2 = pd.concat([df2_1, df2_1_2, df2_2])
survey3 = pd.concat([df3_1])
survey1['survey']=np.full(survey1.shape[0], 1)
survey2['survey']=np.full(survey2.shape[0], 2)
survey3['survey']=np.full(survey3.shape[0], 3)

main = pd.concat([survey1, survey2, survey3])
main

,Date (Responses to recipient forms),Recipient,Origin (Responses to recipient forms),Score (Responses to recipient forms),Primary key,Conditional: Portal Engagement Explanation,Caregiver for Active Duty Military (Constituent Type),Caregiver for Veteran (Constituent Type),Other caregiver (Constituent Type),Veteran (Constituent Type),Other (Constituent Type),Sex,Age,Race:,Time Providing Care,Use of Video Conferencing,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Hypertension (high blood pressure) (Chronic Conditions),Hyperlipidemia (high blood cholesterol or triglyceride levels) (Chronic Conditions),"Allergies, sinusitis and other upper respiratory conditions (Chronic Conditions)",Arthritis (Chronic Conditions),Mood Disorder (depression and bipolar disorder) (Chronic Conditions),Diabetes (Type1 and Type 2) (Chronic Conditions),Anxiety Disorder (Chronic Conditions),Asthma (Chronic Conditions),Coronary artery disease (includes myocardial infarction/heart attack) (Chronic Conditions),Thyroid disorder (Chronic Conditions),Chronic obstructive lung disease and bronchiectasis (Chronic Conditions),Traumatic Brain Injury (TBI) (Chronic Conditions),Post-Traumatic Stress Disorder (PTSD) (Chronic Conditions),None of the above (Chronic Conditions),Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,Consent,Flow,Display and Format,Content,survey
0,2020/12/04 10:55:54,Evans Melanie (mryumyumsbakery@gmail.com),NaN,15,397637986,NaN,0,1,0,0,0,Female,45 - 54,White,5+ years,1+ Year,Yes,No,Neutral,Agree,Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Strongly Disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Rarely,Sometimes,Never,Never,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,1,0,0,1,0,0,1,1,0,1,0,0,0,0,1,1,1,0,0,1,1,1,0,Agree,1,5,5,5,1
1,2020/12/04 10:56:29,Miranda Jennifer (jmiranda26@icloud.com),NaN,14,397628088,NaN,0,1,0,0,0,Female,35 - 44,White,1 to 2 years,<3 Months,No,No,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Disagree,Strongly Disagree,Strongly Disagree,Disagree,Strongly Disagree,Disagree,Agree,Agree,Agree,Neutral,Sometimes,Sometimes,Sometimes,Rarely,Never,Never,Never,Never,Rarely,Rarely,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1,1,0,0,Agree,1,5,5,4,1
2,2020/12/04 10:56:42,Zuniga Adam (zuniga.adam77@gmail.com),NaN,15,397642487,NaN,0,0,0,1,0,Male,35 - 44,White,5+ years,3 To 6 Months,No,No,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Strongly Agree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Neutral,Agree,Agree,Never,Never,Rarely,Never,Never,Never,Never,Never,Never,Never,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,Agree,1,5,5,5,1
3,2020/12/04 10:57:02,Black Thomas (andrearuszala@gmail.com),NaN,12,397637989,NaN,0,0,0,1,0,Male,65+,White,6mo to 1year,6 To 9 Months,Yes,Yes,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Agree,Agree,Agree,Agree,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,Agree,1,4,4,4,1
4,2020/12/04 10:57:09,Jesseapontejr@Gmail.Com Jesse (jesseapontejr@g...,NaN,15,397642488,NaN,0,0,0,1,0,Male,35 - 44,Black or African American,2 to 3 years,1+ Year,

In [24]:
print(survey1.shape)
print(survey2.shape)
print(survey3.shape)

(702, 71)
(605, 71)
(12, 71)


In [25]:
toDrop = ['Date (Responses to recipient forms)', 'Age', 'Recipient', 'Origin (Responses to recipient forms)', 'Primary key', 'Conditional: Portal Engagement Explanation', 'Consent', 'Flow', 'Display and Format', 'Content', 'Sex', 'Race:', 'Time Providing Care', ]
main = main.drop(columns = toDrop)
#main.drop(list(main.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
main.drop(list(main.filter(regex = 'Chronic Conditions')), axis = 1, inplace = True)
#main.drop(list(dropped_dfmain.filter(regex = 'Portal Video Use')), axis = 1, inplace = True)
#main.drop(list(dropped_dfmain.filter(regex = 'Connection Technology Use')), axis = 1, inplace = True)

In [35]:
copy = main.copy()
copy["Caregiver?"] = copy["Caregiver for Active Duty Military (Constituent Type)"] + copy["Caregiver for Veteran (Constituent Type)"] + copy["Other caregiver (Constituent Type)"]
copy = copy.sort_values(by = "Caregiver?")
copy = copy.reset_index(drop = True)
caregiverthing = copy["Caregiver?"]
counter = 0
while caregiverthing[counter] == 0:
    counter += 1
print (counter)
copy = copy.drop(columns = 'Score (Responses to recipient forms)')
copy.drop(list(copy.filter(regex = 'Constituent Type')), axis = 1, inplace = True)
copy = copy.drop(columns = "Caregiver?")
caregivers = copy.drop(copy.index[0:counter])
notCaregivers = copy.drop(copy.index[counter:len(copy.index)+1])
notCaregivers

481


,Use of Video Conferencing,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,6 To 9 Months,Yes,Yes,Agree,Agree,Disagree,Neutral,Agree,Agree,Agree,Agree,Neutral,Neutral,Strongly Disagree,Disagree,Strongly Disagree,Strongly Disagree,Sometimes,Sometimes,Rarely,Sometimes,Rarely,Sometimes,Rarely,Rarely,Rarely,Rarely,0,0,0,1,0,0,0,0,1,Strongly Disagree,3
1,1+ Year,Yes,Yes,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,1,1,1,0,0,1,1,1,0,Strongly Agree,2
2,6 To 9 Months,Yes,Yes,Disagree,Strongly Agree,Strongly Agree,Neutral,Strongly Disagree,Strongly Agree,Strongly Agree,Neutral,Strongly Disagree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Sometimes,Sometimes,Sometimes,Sometimes,Rarely,Sometimes,Sometimes,Sometimes,Sometimes,Sometimes,0,1,0,0,0,0,1,1,0,Disagree,1
3,6 To 9 Months,No,No,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Agree,Strongly Disagree,Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Strongly Disagree,Neutral,Neutral,Never,Never,Never,Never,Sometimes,Never,Never,Never,Rarely,Rarely,0,1,1,1,0,0,0,0,1,Neutral,1
4,3 To 6 Months,No,No,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Disagree,Neutral,Strongly Disagree,Neutral,Agree,Neutral,Agree,Agree,Rarely,Rarely,Never,Rarely,Rarely,Never,Rarely,Rarely,Sometimes,Sometimes,0,1,0,0,0,0,0,1,0,Disagree,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,<3 Months,Yes,Yes,Neutral,Strongly Agree,Disagree,Strongly Agree,Strongly Agree,Neutral,Neutral,Strongly Agree,Neutral,Strongly Agree,Neutral,Agree,Agree,Neutral,Sometimes,Always,Rarely,Always,Never,Rarely,Always,Never,Always,Always,1,1,1,0,0,0,0,0,1,Neutral,1
477,6 To 9 Months,No,No,Disagree,Agree,Neutral,Agree,Neutral,Disagree,Neutral,Neutral,Neutral,Neutral,Agree,Agree,Agree,Agree,Rarely,Rarely,Sometimes,Rarely,Sometimes,Rarely,Rarely,Rarely,Sometimes,Rarely,0,0,0,0,1,1,1,1,0,Agree,2
478,<3 Months,Yes,Yes,Neutral,Agree,Agree,Agree,Neutral,Agree,Neutral,Neutral,Disagree,Strongly Disagree,Agree,Agree,Agree,Agree,Sometimes,Rarely,Never,Sometimes,Never,Never,Rarely,Never,Rarely,Never,0,1,1,0,0,1,1,1,0,Agree,1
479,9 To 12 Months,Yes,Yes,Agree,Agree,Agree,Agree,Agree,Strongly Agree,Strongly Agree,Neutral,Neutral,Strongly Agree,Strongly Agree,Agree,Agree,Agree,Always,Sometimes,Always,Sometimes,Always,Always,Sometimes,Sometimes,Always,Always,1,1,1,1,0,1,1,1,0,Agree,1


In [36]:
encoding = {
    'Agree': 1,
    'StronglyAgree': 2,
    'Strongly Agree': 2,
    'StronglyDisagree': -2,
    'Strongly Disagree': -2,
    'Disagree': -1,
    'Neutral': 0,
    'Yes': 1,
    'No': -1,
    'Rarely':-1,
    'Never':-2,
    'Sometimes':1,
    'Always':2,
    '<6mo':-3,
    '6mo to 1year':-2,
    '1 to 2 years': -1,
    '2 to 3 years': 0,
    '3 to 4 years':1,
    '4-5 years': 2,
    '5+ years': 3, 
    '<3 Months': -2,
    '3 To 6 Months': -1,
    '6 To 9 Months': 0,
    '9 To 12 Months': 1,
    '1+ Year': 2,
    '0 - 17': -3,
    '18 - 24': -2,
    '25 - 34': -1,
    '35 - 44': 0,
    '45 - 54': 1,
    '55 - 64': 2,
    '65+': 3,
    'Male': 0,
    'Female': 1,
    'White': -2, 
    'Black or African American': -1,
    'Asian': 0,
    'American Indian or Alaska Native': 1,
    'Some other race': 2,
    'Native Hawaiian and Pacific Islander': 3  
}

def f(s):
    l = []
    for b in s:
        if (b in encoding):
            l.append(encoding[b])
        elif isinstance(b, str) and b.isnumeric():
            l.append(int(b))
        else:
            l.append(b)
    return pd.Series(l)


In [37]:
data1 = caregivers.apply(f, axis = 0)
data1 = data1.sample(frac=1).reset_index(drop=True)
data2 = notCaregivers.apply(f, axis = 0)
data2 = data2.sample(frac=1).reset_index(drop=True)
data2

,Use of Video Conferencing,Depression,Motivation,Life Satisfaction,Physical Exhaustion,Desire To Escape,Same Self,Financial Decrease,Health Change,Demands Strength,Competing Demand,Concern About Future,Relationship Impact,Portal Experience,Portal Engagement,Portal Engagement Reinforcement,Portal Wellbeing Influence,Unhappy Alone,No One To Talk To,Cant Tolerate Being Alone,No One Understands Me,Waiting for Contact,Feel Completely Alone,Unable To Communicate,Need Company,Difficult to make friends,Feel Excluded,Caregivers (Portal Video Use),Family members (Portal Video Use),Friends (Portal Video Use),Others (Portal Video Use),Not using Portal video (Portal Video Use),SMS/text (Connection Technology Use),Phone (Connection Technology Use),Video Portal (Connection Technology Use),Other (Connection Technology Use),Portal Chronic Disease Management Help,survey
0,-2,-1,-1,-1,-1,-1,-1,0,0,1,1,-1,-1,1,1,1,1,1,-1,-1,-1,-1,-1,1,-1,-1,-2,1,1,1,0,0,0,1,1,0,1,2
1,-2,1,1,-1,1,2,2,2,-1,0,1,0,-2,2,2,2,1,2,-1,2,1,1,1,2,-1,2,2,0,1,0,1,0,1,1,0,1,2,2
2,0,1,1,-1,0,-2,-2,-2,-2,0,-1,-1,-1,1,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,0,1,1,1,0,1,2
3,2,1,1,1,1,1,1,1,2,2,1,2,2,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,2,2
4,1,1,1,0,1,-1,1,-1,0,0,0,-1,0,1,1,1,0,1,-2,1,1,1,1,-1,1,1,1,0,1,1,0,0,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,1,-1,-1,-2,-2,-1,-1,0,-1,2,-1,-1,-1,2,2,2,2,-1,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,1,1,0,1,1,1,0,2,1
477,2,1,1,0,2,2,2,0,1,1,1,0,1,0,0,0,0,2,2,1,2,1,1,1,-1,2,1,1,1,0,0,0,1,1,1,0,0,2
478,2,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,1,0,1,1,-1,-1,-2,-1,-1,-2,-2,-1,1,-1,0,1,1,1,0,1,1,1,0,1,1
479,2,-1,1,0,0,1,-1,1,-1,0,0,0,0,-1,1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,0,1,0,0,0,0,0,0,1,2,1


In [38]:
ls = []
for column in data2:
    ls.append(data2[column].unique())
ls

[array([-2,  0,  2,  1, -1], dtype=int64),
 array([-1,  1], dtype=int64),
 array([-1,  1], dtype=int64),
 array([-1,  1,  0,  2, -2], dtype=int64),
 array([-1,  1,  0,  2, -2], dtype=int64),
 array([-1,  2, -2,  1,  0], dtype=int64),
 array([-1,  2, -2,  1,  0], dtype=int64),
 array([ 0,  2, -2,  1, -1], dtype=int64),
 array([ 0, -1, -2,  2,  1], dtype=int64),
 array([ 1,  0,  2, -1, -2], dtype=int64),
 array([ 1, -1,  0,  2, -2], dtype=int64),
 array([-1,  0,  2,  1, -2], dtype=int64),
 array([-1, -2,  2,  0,  1], dtype=int64),
 array([ 1,  2,  0, -1, -2], dtype=int64),
 array([ 1,  2,  0, -2, -1], dtype=int64),
 array([ 1,  2,  0, -1, -2], dtype=int64),
 array([ 1,  0,  2, -1, -2], dtype=int64),
 array([ 1,  2, -1, -2], dtype=int64),
 array([-1,  1, -2,  2], dtype=int64),
 array([-1,  2,  1, -2], dtype=int64),
 array([-1,  1, -2,  2], dtype=int64),
 array([-1,  1, -2,  2], dtype=int64),
 array([-1,  1, -2,  2], dtype=int64),
 array([ 1,  2, -1, -2], dtype=int64),
 array([-1,  1, -2, 

In [39]:
import statsmodels.stats.weightstats as ssw

#results = []

#pd.set_option('display.max_rows', 500)
#for col in data1.columns: 
#    vals1 = data1[col].to_list()
#    vals2 = data2[col].to_list()
#    
#    # Null hypothesis: means are the same
#    # Alternative: the mean after is higher
#    
#    cm = ssw.CompareMeans(ssw.DescrStatsW(vals1), ssw.DescrStatsW(vals2))
#    res = cm.ztest_ind(alternative='smaller',usevar='unequal')
#    msg = "Cannot reject the null"
#    if (res[1] <= 0.05):
#        msg = "Alternative accepted: mean is larger"
#    elif (res[1] >= 0.95):
#        msg = "Alternative accepted: mean is smaller"
#    results.append([col, res[1], msg])
#
#res = pd.DataFrame(results, columns=['Item', 'P-Value', 'Verdict'])
#res

In [40]:
def analyze(before, after):
    results = []
    for col in before.columns: 
        vals1 = before[col].to_list()
        vals2 = after[col].to_list()

        # Null hypothesis: means are the same
        # Alternative: the mean after is higher

        cm = ssw.CompareMeans(ssw.DescrStatsW(vals1), ssw.DescrStatsW(vals2))
        res = cm.ztest_ind(alternative='smaller',usevar='unequal')
        msg = "Cannot reject the null"
        if (res[1] <= 0.05):
            msg = "Alternative accepted: mean is larger"
            results.append([col, res[1], msg])
        elif (res[1] >= 0.95):
            msg = "Alternative accepted: mean is smaller"
            results.append([col, 1-res[1], msg])
        else:
            results.append([col, res[1], msg])
    res = pd.DataFrame(results, columns=['Item', 'P-Value', 'Verdict'])
    return res

In [41]:
w1data1 = data1[data1['survey']==1]
w2data1 = data1[data1['survey']==2]
w1data2 = data2[data2['survey']==1]
w2data2 = data2[data2['survey']==2]

print(w1data1.shape)
print(w2data1.shape)
print(w1data2.shape)
print(w2data2.shape)

survey1totals = pd.concat([w1data1, w1data2])
survey2totals = pd.concat([w2data1, w2data2])

print(survey1totals.shape)
print(survey2totals.shape)
print(survey2totals.sum(axis = 0))
results = []

(429, 38)
(399, 38)
(273, 38)
(206, 38)
(702, 38)
(605, 38)
Use of Video Conferencing                     -22
Depression                                    103
Motivation                                     75
Life Satisfaction                             -59
Physical Exhaustion                           426
Desire To Escape                               82
Same Self                                     281
Financial Decrease                            161
Health Change                                 159
Demands Strength                              392
Competing Demand                              173
Concern About Future                           27
Relationship Impact                            96
Portal Experience                             556
Portal Engagement                             492
Portal Engagement Reinforcement               479
Portal Wellbeing Influence                    471
Unhappy Alone                                 299
No One To Talk To                       

In [42]:
res = analyze(w1data1, w1data2)
res

c:\users\14058\appdata\local\programs\python\python38\lib\site-packages\statsmodels\stats\weightstats.py:746: RuntimeWarning: invalid value encountered in double_scalars
  zstat = (value1 - value2 - diff) / std_diff


,Item,P-Value,Verdict
0,Use of Video Conferencing,5.687674e-03,Alternative accepted: mean is smaller
1,Depression,6.386743e-06,Alternative accepted: mean is larger
2,Motivation,1.003483e-07,Alternative accepted: mean is larger
3,Life Satisfaction,4.442009e-05,Alternative accepted: mean is smaller
4,Physical Exhaustion,4.091408e-02,Alternative accepted: mean is larger
5,Desire To Escape,6.154437e-03,Alternative accepted: mean is larger
6,Same Self,6.293072e-05,Alternative accepted: mean is larger
7,Financial Decrease,8.446745e-01,Cannot reject the null
8,Health Change,9.456415e-01,Cannot reject the null
9,Demands Strength,9.450235e-01,Cannot reject the null


In [43]:
res = analyze(w2data1, w2data2)
res

c:\users\14058\appdata\local\programs\python\python38\lib\site-packages\statsmodels\stats\weightstats.py:746: RuntimeWarning: invalid value encountered in double_scalars
  zstat = (value1 - value2 - diff) / std_diff


,Item,P-Value,Verdict
0,Use of Video Conferencing,0.698636,Cannot reject the null
1,Depression,0.000918,Alternative accepted: mean is larger
2,Motivation,0.000007,Alternative accepted: mean is larger
3,Life Satisfaction,0.000001,Alternative accepted: mean is smaller
4,Physical Exhaustion,0.442853,Cannot reject the null
5,Desire To Escape,0.002093,Alternative accepted: mean is larger
6,Same Self,0.000276,Alternative accepted: mean is larger
7,Financial Decrease,0.008332,Alternative accepted: mean is smaller
8,Health Change,0.835952,Cannot reject the null
9,Demands Strength,0.000235,Alternative accepted: mean is smaller


In [44]:
res = analyze(w1data1, w2data1)
res

c:\users\14058\appdata\local\programs\python\python38\lib\site-packages\statsmodels\stats\weightstats.py:746: RuntimeWarning: divide by zero encountered in double_scalars
  zstat = (value1 - value2 - diff) / std_diff


,Item,P-Value,Verdict
0,Use of Video Conferencing,5.916487e-10,Alternative accepted: mean is smaller
1,Depression,2.254080e-01,Cannot reject the null
2,Motivation,5.786121e-01,Cannot reject the null
3,Life Satisfaction,3.895246e-03,Alternative accepted: mean is larger
4,Physical Exhaustion,8.649512e-03,Alternative accepted: mean is larger
5,Desire To Escape,1.156048e-01,Cannot reject the null
6,Same Self,1.054705e-01,Cannot reject the null
7,Financial Decrease,1.910654e-02,Alternative accepted: mean is larger
8,Health Change,1.548388e-02,Alternative accepted: mean is larger
9,Demands Strength,5.037757e-05,Alternative accepted: mean is larger


In [45]:
res = analyze(w1data2, w2data2)
res

c:\users\14058\appdata\local\programs\python\python38\lib\site-packages\statsmodels\stats\weightstats.py:746: RuntimeWarning: divide by zero encountered in double_scalars
  zstat = (value1 - value2 - diff) / std_diff


,Item,P-Value,Verdict
0,Use of Video Conferencing,7.436174e-04,Alternative accepted: mean is smaller
1,Depression,5.709211e-01,Cannot reject the null
2,Motivation,6.799056e-01,Cannot reject the null
3,Life Satisfaction,2.168560e-01,Cannot reject the null
4,Physical Exhaustion,3.449518e-01,Cannot reject the null
5,Desire To Escape,7.389975e-02,Cannot reject the null
6,Same Self,1.720985e-01,Cannot reject the null
7,Financial Decrease,4.635020e-01,Cannot reject the null
8,Health Change,2.478632e-02,Alternative accepted: mean is larger
9,Demands Strength,2.102073e-01,Cannot reject the null


In [ ]:
pd.set_option('display.max_rows', 100)
data.sort_values(by=['survey'])

In [ ]:
#toDrop2 = ['Depression', 'Motivation', 'Physical Exhaustion', 'Desire To Escape', 'Same Self', 'Financial Decrease', 'Health Change', 'Demands Strength', 'Competing Demand', 'Concern About Future', 'Relationship Impact', 'Unhappy Alone', 'No One To Talk To', 'Cant Tolerate Being Alone', 'No One Understands Me', 'Waiting for Contact', 'Feel Completely Alone', 'Unable To Communicate', 'Need Company', 'Difficult to make friends', 'Feel Excluded']
toDrop3 = ['Score (Responses to recipient forms)']
data1 = data.drop(columns = toDrop2)
data1 = data.drop(columns = toDrop3)
data1.shape

In [ ]:
test1 = data[data['survey']==1]['Use of Video Conferencing']
test2 = data[data['survey']==2]['Use of Video Conferencing']
print(test1.sum()/test1.shape[0])
print(test2.sum()/test2.shape[0])

In [ ]:
train_size = int(data1.shape[0]*0.8)
test_size = int(data1.shape[0]*0.2)

In [ ]:
#creates Bayesian model using data

dtrain = data1[:train_size]
dtest = data1[:-test_size]

In [ ]:
model = BayesianNetwork.from_samples(dtrain)

In [ ]:
blanked = dtest.copy()
blanked['Score (Responses to recipient forms)'] = np.full(blanked.shape[0], None)
prediction = model.predict(blanked.values.tolist())
mean = dtest["Score (Responses to recipient forms)"].mean()
rss = 0
tss = 0
correct = 0
for i in range (0, dtest.shape[0]):
    yBar = prediction[i][0]
    
    yActual = dtest['Score (Responses to recipient forms)'][i]
    
    if (yBar == yActual):
        correct+=1
    
    rss += (yActual-yBar)**2
    tss += (yActual-mean)**2

percent = float(correct) / dtest.shape[0]
r2 = 1-float(rss)/tss
print("Accuracy: " + (str)(percent))
print("r^2: " + (str)(r2))


from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score

labels = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
y_true = dtest['Score (Responses to recipient forms)']
y_pred = [row[0] for row in prediction]

precision = precision_score(y_true, y_pred, labels=labels, average='micro')
recall = recall_score(y_true, y_pred, labels=labels, average='micro')
accuracy = accuracy_score(y_true, y_pred)
# Need to make y_true shape (samples, classes)

#y_true = y_true.values.reshape((y_true.shape[0],len(labels)))
#y_pred = y_pred.reshape((y_pred.shape[0],len(labels)))

r_squared = r2_score(y_true, y_pred)
#auc = roc_auc_score(y_true, y_pred)

print('Accuracy: %.3f' % accuracy)
print('R^2: %.3f' % r_squared)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
#print('ROC AUC: %.3f' % auc)

In [ ]:
blanked = dtest.copy()
blanked['Life Satisfaction'] = np.full(blanked.shape[0], None)
prediction = model.predict(blanked.values.tolist())
mean = dtest["Life Satisfaction"].mean()
rss = 0
tss = 0
correct = 0
for i in range (0, dtest.shape[0]):
    yBar = prediction[i][0]
    
    yActual = dtest['Life Satisfaction'][i]
    print(yBar)
    if (yBar == yActual):
        correct+=1
    
    rss += (yActual-yBar)**2
    tss += (yActual-mean)**2

percent = float(correct) / dtest.shape[0]
r2 = 1-float(rss)/tss
print("Accuracy: " + (str)(percent))
print("r^2: " + (str)(r2))


from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score

labels = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
y_true = dtest['Life Satisfaction']
y_pred = [row[0] for row in prediction]

precision = precision_score(y_true, y_pred, labels=labels, average='micro')
recall = recall_score(y_true, y_pred, labels=labels, average='micro')
accuracy = accuracy_score(y_true, y_pred)
# Need to make y_true shape (samples, classes)

#y_true = y_true.values.reshape((y_true.shape[0],len(labels)))
#y_pred = y_pred.reshape((y_pred.shape[0],len(labels)))

r_squared = r2_score(y_true, y_pred)
#auc = roc_auc_score(y_true, y_pred)

print('Accuracy: %.3f' % accuracy)
print('R^2: %.3f' % r_squared)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
#print('ROC AUC: %.3f' % auc)

In [ ]:
from pymc3.glm import GLM

with pm.Model() as model_glm:
    GLM.from_formula("Life_Satisfaction ~ Caregiver_for_Veteran + Other_caregiver + Veteran + Other + Hypertension_high_blood_pressure_Chronic_Conditions + Hyperlipidemia_high_blood_cholesterol_or_triglyceride_levels_Chronic_Conditions + Allergies + sinusitis_and_other_upper_respiratory_conditions_Chronic_Conditions + Arthritis_Chronic_Conditions + Mood_Disorder_depression_and_bipolar_disorder_Chronic_Conditions + Diabetes_Type1_and_Type_2_Chronic_Conditions + Anxiety_Disorder_Chronic_Conditions + Asthma_Chronic_Conditions + Coronary_artery_disease_includes_myocardial_infarction_heart_attack_Chronic_Conditions + Thyroid_disorder_Chronic_Conditions + Chronic_obstructive_lung_disease_and_bronchiectasis_Chronic_Condition + Traumatic_Brain_Injury__TBI_Chronic_Conditions + Post_Traumatic_Stress_Disorder_PTSD_Chronic_Conditions + None_of_the_above__Chronic_Conditions + Caregivers_Portal_Video_Use + Family_members__Portal_Video_Us + Friends_Portal_Video_Use + Others_Portal_Video_Use + Not_using_Portal_video_Portal_Video_Use + Portal_Chronic_Disease_Management_Help + SMS_text_Connection_Technology_Use + Phone_Connection_Technology_Use + Video_Portal_Connection_Technology_Use + Other_Connection_Technology_Use + before", dtrain)
    trace = pm.sample()

In [ ]:
with model_glm:
    axes_arr = az.plot_trace(trace)
plt.draw()

In [ ]:
blanked = dtest.copy(dtest)
blanked['Life_Satisfaction'] = np.full(blanked.shape[0])
prediction = model.predict(blanked)
mean = dtest["weight"].mean()
rss = 0
tss = 0
correct = 0
for i in range (0, dtest.shape[0]):
    yBar = prediction['Life_Satisfaction'][i]
    yActual = dtest['Life_Satisfaction'][i]
    if (yBar == yActual):
        correct+=1
    rss += (yActual-yBar)**2
    tss += (yActual-mean)**2
percent = (float)correct/200
r2 = 1-(float)rss/tss
print("Accuracy: " + (str)(percent))
print("r^2: " + (str)(r2))

from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score

labels = [-2, -1, 0, 1, 2]
y_true = dtest['Life_Satisfaction']
y_pred = prediction['Life_Satisfaction']

precision = precision_score(y_true, y_pred, labels=labels, average='micro')
recall = recall_score(y_true, y_pred, labels=labels, average='micro')
accuracy = accuracy_score(y_true, y_pred)
# Need to make y_true shape (samples, classes)

y_true = y_true.reshape((y_true.shape[0],len(labels)))
y_pred = y_pred.reshape((y_pred.shape[0],len(labels)))

r_squared = r2_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred)

print('Accuracy: %.3f' % accuracy)
print('R^2: %.3f' % r_squared)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
print('ROC AUC: %.3f' % auc)